<a href="https://colab.research.google.com/github/mzkhan2000/SA/blob/main/Word2VecEmbeggings_UsingWordVector_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:
import gensim.downloader as api

In [86]:
from scipy import spatial
import pandas as pd
import numpy as np
import re
from collections import OrderedDict, defaultdict
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [85]:
word_vectors = api.load("glove-wiki-gigaword-200")

[==================================================] 100.0% 252.1/252.1MB downloaded


In [146]:
from google.colab import files
uploaded = files.upload()

Saving sections_titles.csv to sections_titles.csv


In [160]:
data = pd.read_csv('sections_titles.csv', header=None)
df = data
df.columns = ["section", "sec_title"]
df["formatted_sec_title"] = df["sec_title"]

In [148]:
# Trasforming texts from the section titles
for i in range(0, len(df)):
  sec_title_text = df.iloc[i]["sec_title"]
  # Removing alpha-numerical and extra Spaces
  # Removing Square Brackets and Extra Spaces
  sec_title_text = re.sub(r'[[0-9]*]', ' ', sec_title_text)
  # sec_title_text = re.sub(r's+', ' ', sec_title_text)
  # Removing special characters and digits
  sec_title_text = re.sub('[^a-zA-Z]', ' ', sec_title_text )
  # sec_title_text = re.sub(r'[^A-Za-z0-9\s]', r'', str(sec_title_text).lower())
  # removing stop words from English
  sec_title_text = " ".join([word for word in sec_title_text.split()
                      if word not in stopwords.words('english')])
  # inserting formatted text values to the dataframe
  df.at[i, "formatted_sec_title"] = sec_title_text

In [161]:
# Trasforming texts from the section titles
for i in range(0, len(df)):
  sec_title_text = df.iloc[i]["sec_title"]
  # Removing Square Brackets and Extra Spaces
  sec_title_text = re.sub(r'[[0-9]*]', ' ', sec_title_text)
  # sec_title_text = re.sub(r's+', ' ', sec_title_text)
  # Removing special characters and digits
  sec_title_text = re.sub('[^a-zA-Z]', ' ', sec_title_text )
  # sec_title_text = re.sub(r's+', ' ', sec_title_text)
  # inserting formatted text values to the dataframe
  df.at[i, "formatted_sec_title"] = sec_title_text


In [162]:
section_list = df["section"].tolist()
book = {key: None for key in section_list}

In [163]:
# Trasforming section titles into tokens and inserting values to the book dictionary
for i in range(0, len(df)):
  sec_name = df.iloc[i]["section"]
  # tokens = df.iloc[i]["sec_title"]
  tokens = df.iloc[i]["formatted_sec_title"]
  tokens = tokens.lower().split()
  # inserting values to the book dictionary
  if sec_name in book:
    book[sec_name]= tokens
    

In [117]:
# a test question from Part2_9A
test_question = "If a registered surveyor does not comply with a notice under this section who engage another registered surveyor to make the correction"
test_question = test_question.lower().split()

In [151]:
# a test question from Part3_10A
test_question = "A person registered pursuant to an entitlement to re-registration"
test_question = test_question.lower().split()

In [154]:
# a test question from Part3_14
test_question = "the Civil and Administrative Tribunal for an administrative review"
test_question = test_question.lower().split()

In [164]:
# a test question from Part3_15
test_question = "the manner and  form in which the register of surveyors is to be maintained"
test_question = test_question.lower().split()

In [183]:
# a test question from Part5_21
test_question = "land surveyor"
test_question = test_question.lower().split()

In [188]:
# a test question from Part5_22
test_question = "advertise that he or she is willing to carry out a mining survey for fee or unless the person is a registered mining surveyor"
test_question = test_question.lower().split()

In [189]:
similarity_list = []
keys_list = list(book)
for i in range(0, len(keys_list)):
  key = keys_list[i]
  sec_title_tokens = book[key]
  similarity = word_vectors.n_similarity(test_question, sec_title_tokens)
  similarity_list.append(similarity)

In [190]:
import heapq
top3_sections = []
max_similarity_value = max(similarity_list)
max_indx = similarity_list.index(max_similarity_value)
top3_sec_val = heapq.nlargest(3,similarity_list)
# for i in range(0, len(top3_sec_val)):
 # indx = similarity_list.index(top3_sec_val[i])
 # top3_sections.append(keys_list[indx])
for item in top3_sec_val:
  indx = similarity_list.index(item)
  top3_sections.append(keys_list[indx])
# print(indx)
# print(similarity_list[indx])
# print(keys_list[indx])
print(top3_sections)
print(similarity_list)

['Part2_9B ', 'Part5_25', 'Part3_16A ']
[0.7798315, 0.83002335, 0.88583183, 0.7710304, 0.7386084, 0.8115214, 0.6649144, 0.94031835, 0.8281949, 0.783913, 0.8332698, 0.39117625, 0.40763178, 0.44907767, 0.7628232, 0.64791083, 0.22736187, 0.89527595, 0.45460883, 0.40993133, 0.88454074, 0.8018267, 0.79223454, 0.7843058, 0.7813169, 0.6231477, 0.7297735, 0.92140687, 0.7723345, 0.7094212, 0.7091836, 0.47178093, 0.824941, 0.3155309, 0.65540797, 0.66077274]


In [179]:
print(df["section"])

0        Part2_4
1        Part2_5
2        Part2_6
3        Part2_7
4        Part2_8
5        Part2_9
6      Part2_9A 
7      Part2_9B 
8      Part2_9C 
9       Part3_10
10    Part3_10A 
11      Part3_11
12      Part3_12
13      Part3_13
14      Part3_14
15      Part3_15
16      Part3_16
17    Part3_16A 
18      Part4_17
19      Part4_18
20      Part4_19
21      Part4_20
22      Part5_21
23      Part5_22
24    Part5_22A 
25      Part5_23
26      Part5_24
27      Part5_25
28      Part5_26
29      Part6_27
30      Part6_28
31    Part6_28A 
32      Part6_29
33      Part6_30
34      Part6_31
35      Part6_32
Name: section, dtype: object


In [75]:
print(df["sec_title"])

0               Surveys carried out by Surveyor General
1             Surveys carried out by public authorities
2     Details of surveys to be provided to Surveyor-...
3                            Register of public surveys
4                       Co-ordination of public surveys
5      Maintenance and repair of permanent survey marks
6                           Correction of survey errors
7     Authority for a registered surveyor to act for...
8     Register of public surveys to include details ...
9     Application to registration of Licensing and R...
10    Re-registration after removal for non-payment ...
11                                       Investigations
12                            Professional incompetence
13                              Professional misconduct
14    Applications to Civil and Administrative Tribu...
15                                Register of surveyors
16                             Evidentiary certificates
17    Certain provisions of this Part apply also